In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = '/kaggle/input/brain-tumor-mri-dataset'

In [ ]:
!pip install scikit-learn
!pip install seaborn
!pip install plotly
!pip install missingno

In [ ]:
import sys
import os
import math
import pathlib
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['figure.dpi'] = 300
%matplotlib inline
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import EfficientNetV2L

from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import *

from PIL import Image, ImageEnhance
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Training')
test_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Testing')
img_height=224
img_width=224

# Image Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
])

# Normalization
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Dataset loading
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    interpolation="bilinear",
    label_mode="int",
    follow_links=False,
)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Visualize the data distribution

In [ ]:
ROOT_DIR = r"/kaggle/input/brain-tumor-mri-dataset"
TRAIN_DIR = os.path.join(ROOT_DIR, 'Training')
TEST_DIR = os.path.join(ROOT_DIR, 'Testing')
assert os.path.isdir(ROOT_DIR) and os.path.isdir(TRAIN_DIR) and os.path.isdir(TEST_DIR)
TUMOR_CLASS = ['meningioma', 'glioma', 'pituitary', 'notumor']
IMAGE_DATA_PATHS = [os.path.join(TRAIN_DIR, tumor_class) for tumor_class in TUMOR_CLASS]
TEST_DATA_PATHS = [os.path.join(TEST_DIR, tumor_class) for tumor_class in TUMOR_CLASS]

In [ ]:
TEST_DATA_PATHS

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in TEST_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in IMAGE_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
def display_sample_images(paths, classes):
    fig, axes = plt.subplots(nrows=1, ncols=len(paths), figsize=(18, 5))
    for i, (path, cls) in enumerate(zip(paths, classes)):
        image_file = os.listdir(path)[1]
        image_path = os.path.join(path, image_file)
        image = plt.imread(image_path, format='grayscale')

        folder_name = os.path.basename(path)
        axes[i].imshow(image, cmap='gray')
        axes[i].set_title(folder_name)
        axes[i].axis('off')
    plt.show()


display_sample_images(IMAGE_DATA_PATHS, TUMOR_CLASS)

# Function to create and train models

In [ ]:
def create_and_train_model(base_model, model_name, train_ds, val_ds, test_ds, num_classes=4, learning_rate=0.0001, epochs=100):
    for layer in base_model.layers[:10]:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = Dropout(0.4)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.inputs, outputs=predictions)
    model.compile(
        optimizer=Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999),
        loss=tf.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'{model_name}.keras', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

    history = model.fit(
        train_ds,
        shuffle=True,
        validation_data=test_ds,
        epochs=epochs,
        callbacks=[early_stop, model_checkpoint, reduce_lr]
    )

    model.save(f'{model_name}.keras')
    evaluation = model.evaluate(val_ds)

    return model, history, evaluation



# Function to plot training and validation metrics

In [ ]:
def plot_metrics(history, model_name):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, acc, 'g', label='Training accuracy')
    plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
    plt.title(f'{model_name}: Training and validation accuracy')
    plt.legend()

    plt.figure(figsize=(10, 5))
    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title(f'{model_name}: Training and validation loss')
    plt.legend()

    plt.show()



# Function to evaluate a model and return accuracy in percentage

In [ ]:
def evaluate_model(model, dataset, model_name):
    evaluation = model.evaluate(dataset, verbose=0)
    accuracy = evaluation[1] * 100 
    print(f"{model_name} Accuracy: {accuracy:.2f}%")
    return accuracy


# InceptionV3 for practice

In [ ]:
# InceptionV3
base_model1 = InceptionV3(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model1, history1, _ = create_and_train_model(base_model1, 'model1_inceptionv3', train_ds, val_ds, test_ds)
plot_metrics(history1, 'InceptionV3')
inceptionv3_accuracy = evaluate_model(model1, val_ds, 'InceptionV3')

# VGG16

In [ ]:
# VGG16
base_model2 = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model2, history2, _ = create_and_train_model(base_model2, 'model2_vgg16', train_ds, val_ds, test_ds)
plot_metrics(history2, 'VGG16')
vgg16_accuracy = evaluate_model(model2, val_ds, 'VGG16')

# ResNet152V2

In [ ]:
# ResNet152V2
base_model3 = ResNet152V2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model3, history3, _ = create_and_train_model(base_model3, 'model3_resnet152v2', train_ds, val_ds, test_ds)
plot_metrics(history3, 'ResNet152V2')
resnet152v2_accuracy = evaluate_model(model3, val_ds, 'ResNet152V2')

# MobileNetV2

In [ ]:
# MobileNetV2
base_model4 = MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model4, history4, _ = create_and_train_model(base_model4, 'model4_mobilenetv2', train_ds, val_ds, test_ds)
plot_metrics(history4, 'MobileNetV2')
mobilenetv2_accuracy = evaluate_model(model4, val_ds, 'MobileNetV2')

# Xception

In [ ]:
# Xception
base_model5 = Xception(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model5, history5, _ = create_and_train_model(base_model5, 'model5_xception', train_ds, val_ds, test_ds)
plot_metrics(history5, 'Xception')
xception_accuracy = evaluate_model(model5, val_ds, 'Xception')

# EfficientNetV2L

In [ ]:
# EfficientNetV2L
base_model6 = EfficientNetV2L(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
model6, history6, _ = create_and_train_model(base_model6, 'model6_efficientnetv2l', train_ds, val_ds, test_ds)
plot_metrics(history6, 'EfficientNetV2L')
efficientnetv2l_accuracy = evaluate_model(model6, val_ds, 'EfficientNetV2L')
